In [1]:
import pandas as pd
import numpy as np

#FOR VISUALIZATION:
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import emoji
import string
from os import path
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
df = pd.read_csv('D:/Semester 3/Temu Kembali Informasi/spotify 300 7 kata/dataset_300.csv')
df.columns = ['docID', 'text', ]
df

,docID,text
0,1,"Great music service, the audio is high quality..."
1,2,Please ignore previous negative rating. This a...
2,3,"This pop-up ""Get the best Spotify experience o..."
3,4,Dear Spotify why do I get songs that I didn't ...
4,5,The player controls sometimes disappear for no...
...,...,...
295,296,It provides best music quality than any other ...
296,297,I've been paying for my premium only for me no...
297,298,Let me stop autoplay when connecting to car bl...
298,299,"I'm sick of it, Spotify, please stop adding so..."


In [3]:
df.isnull().sum()

docID    0
text     0
dtype: int64

In [4]:
df = df.dropna(subset=['text']) 

In [5]:
def clean_text(text):
    text = re.sub(r'Ã[\x80-\xBF]+', ' ', text) 
    text = re.sub(r'[^a-zA-Z\s]', ' ', text) 
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text.lower()  # Convert all to lowercase

In [6]:
df.docID = pd.Series(["D" + str(ind) for ind in df.docID])
df

,docID,text
0,D1,"Great music service, the audio is high quality..."
1,D2,Please ignore previous negative rating. This a...
2,D3,"This pop-up ""Get the best Spotify experience o..."
3,D4,Dear Spotify why do I get songs that I didn't ...
4,D5,The player controls sometimes disappear for no...
...,...,...
295,D296,It provides best music quality than any other ...
296,D297,I've been paying for my premium only for me no...
297,D298,Let me stop autoplay when connecting to car bl...
298,D299,"I'm sick of it, Spotify, please stop adding so..."


In [7]:
df

,docID,text
0,D1,"Great music service, the audio is high quality..."
1,D2,Please ignore previous negative rating. This a...
2,D3,"This pop-up ""Get the best Spotify experience o..."
3,D4,Dear Spotify why do I get songs that I didn't ...
4,D5,The player controls sometimes disappear for no...
...,...,...
295,D296,It provides best music quality than any other ...
296,D297,I've been paying for my premium only for me no...
297,D298,Let me stop autoplay when connecting to car bl...
298,D299,"I'm sick of it, Spotify, please stop adding so..."


In [8]:
print(df.iloc[283]['text']) 

Nice adio quality but shoul try to provide better quality for free


In [9]:
df['text'] = df['text'].apply(clean_text) 

In [10]:
df.to_csv('pre1.csv',index=False)

In [11]:
def tokenize_text(text):
    tokens = word_tokenize(text)  
    return tokens  

df['text'] = df['text'].apply(tokenize_text)

print(df.iloc[283]['text']) 

['nice', 'adio', 'quality', 'but', 'shoul', 'try', 'to', 'provide', 'better', 'quality', 'for', 'free']


In [12]:
df.to_csv('tokenisasi.csv', index=False)

In [13]:
# Menghapus stopwords
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))  # Get the set of English stopwords
    if custom_stopwords:
        stop_words.update(custom_stopwords)
    return [word for word in tokens if word.lower() not in stop_words]  # Filter out stopwords

In [14]:
custom_stopwords = {'app', 'music','play', 'spotify', 'song', 'songs', 'listen', 'playing','get', 'playlist'}
df['text'] = df['text'].apply(remove_stopwords)

In [15]:
df.head()

,docID,text
0,D1,"[great, service, audio, high, quality, easy, u..."
1,D2,"[please, ignore, previous, negative, rating, s..."
2,D3,"[pop, best, experience, android, annoying, ple..."
3,D4,"[dear, put, shuffle]"
4,D5,"[player, controls, sometimes, disappear, reaso..."


In [16]:
df.to_csv('stopword.csv', index=False)

In [17]:
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to perform lemmatization
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(word, pos='v') for word in tokens]
#pos is set to v (verb) for better accuracy. Refer below manual to understand more.
# refer: https://www.nltk.org/book/ch05.html
df['text'] = df['text'].apply(lemmatize_tokens)

In [18]:
df.to_csv('stemming.csv', index=False)

In [50]:
df.head()

,docID,text
0,D1,"[great, service, audio, high, quality, easy, u..."
1,D2,"[please, ignore, previous, negative, rat, supe..."
2,D3,"[pop, best, experience, android, annoy, please..."
3,D4,"[dear, put, shuffle]"
4,D5,"[player, control, sometimes, disappear, reason..."


In [19]:
# Inisialisasi inverted index
inverted_index = {}

# Iterasi melalui dataframe
for index, row in df.iterrows():
    doc_id = row['docID']
    words = row['text']
    
    for word in words:
        if word not in inverted_index:
            inverted_index[word] = set()  # Buat set baru untuk kata ini
        inverted_index[word].add(doc_id)  # Tambahkan docID ke set

# Menampilkan hasil inverted index
for word, doc_ids in inverted_index.items():
    print(f"{word}: {', '.join(doc_ids)}")


great: D219, D132, D193, D257, D252, D268, D264, D209, D56, D114, D238, D148, D39, D171, D244, D165, D1, D38, D287, D280, D292, D8, D236, D249, D2, D229, D184, D230
service: D40, D142, D20, D9, D175, D1, D115, D280
audio: D1, D94, D62, D262
high: D1, D117, D264
quality: D40, D55, D296, D117, D181, D1, D284, D260, D288
easy: D209, D141, D294, D192, D1, D118, D290, D81, D95, D288, D31
use: D129, D228, D79, D29, D170, D205, D259, D141, D207, D64, D221, D16, D134, D80, D209, D168, D146, D106, D203, D216, D68, D270, D293, D300, D47, D156, D100, D167, D148, D73, D26, D294, D155, D192, D58, D17, D124, D145, D176, D175, D225, D1, D74, D87, D173, D199, D280, D279, D288, D84, D72, D276, D292, D85, D8, D232, D81, D236, D295, D97, D229
also: D275, D272, D72, D180, D291, D146, D176, D143, D1, D298, D232, D183, D258
quick: D1, D266
friendly: D1, D280, D202
support: D197, D194, D1, D48, D258
please: D289, D132, D205, D187, D126, D52, D64, D7, D262, D103, D163, D76, D281, D190, D299, D124, D173, D22, 

In [20]:
# Langkah 1: Mengonversi inverted index menjadi list of tuples
inverted_index_list = [(word, ', '.join(doc_ids)) for word, doc_ids in inverted_index.items()]

# Langkah 2: Membuat DataFrame
inverted_index_df = pd.DataFrame(inverted_index_list, columns=['Word', 'DocIDs'])

# Langkah 3: Menyimpan ke CSV
inverted_index_df.to_csv('inverted_index.csv', index=False)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Membuat daftar dari teks dokumen
documents = df['text'].apply(lambda x: ' '.join(x)).tolist()

# Langkah 1: Membuat matriks frekuensi kata
vectorizer = TfidfVectorizer(use_idf=False, norm=None, binary=False)
word_count_matrix = vectorizer.fit_transform(documents).toarray()

# Membuat DataFrame frekuensi kata
word_freq_df = pd.DataFrame(word_count_matrix, columns=vectorizer.get_feature_names_out())
word_freq_df['docID'] = df['docID']

# Mengatur ulang DataFrame agar docID berada di kolom pertama
word_freq_df = word_freq_df.set_index('docID')

In [22]:
# Menyimpan DataFrame ke CSV
word_freq_df.to_csv('word_frequency.csv')

In [23]:
# Langkah 2: Menghitung nilai TF-IDF
tfidf_vectorizer = TfidfVectorizer(use_idf=True, norm='l2')
tfidf_matrix = tfidf_vectorizer.fit_transform(documents).toarray()

# Membuat DataFrame untuk TF-IDF
tfidf_df = pd.DataFrame(tfidf_matrix, columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df['docID'] = df['docID']

# Mengatur ulang DataFrame agar docID berada di kolom pertama
tfidf_df = tfidf_df.set_index('docID')

# Menggabungkan frekuensi kata dan TF-IDF
final_df = word_freq_df.join(tfidf_df, rsuffix='_tfidf')

# Menyimpan ke CSV
final_df.to_csv('word_freq_tfidf.csv')